In this notebook, we're going to talk about [`matplotlib`](https://matplotlib.org/) — a comprehensive library for creating static, animated, and interactive visualizations in Python. We will use this library throughout the course to visualize our datasets and the performance of the machine learning models we will eventually be building.

It should be noted that `pandas` has many built-in functions for generating plots directly from `DataFrames`, typically using `matplotlib` behind the scenes. While this can sometimes be more convenient than grabbing data yourself and then mantually writing `matplotlib` code to generate a figure, there are going to be many situations when you are going to want to plot data from `numpy` arrays or other data structures. Therefore, we're going to stick with `matplotlib` so that you have the tools you need to plot anything.

In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt

# Important: Run this code cell each time you start a new session!

In [ ]:
!pip install numpy
!pip install pandas
!pip install os
import numpy as np
import pandas as pd
import os

In [ ]:
!wget -nc https://archive.physionet.org/users/shared/challenge-2019/training_setA.zip
!unzip -n training_setA.zip

In [ ]:
def load_single_file(file_path):
    df = pd.read_csv(file_path, sep="|")
    df['PatientID'] = file_path.split(os.sep)[-1][:-4]
    df['Hour'] = df.index
    keep_cols = ['PatientID', 'Age', 'Gender', 'SepsisLabel', 'Hour',
                 'HR', 'O2Sat', 'SBP', 'DBP', 'Resp']
    df = df[keep_cols]
    df.rename(columns={'Gender': 'Sex', 'SepsisLabel': 'HasSepsis'}, inplace=True)
    return df

def create_final_table():
    final_df = pd.DataFrame()
    patient_list = range(1, 11)
    patient_list = [f'p{str(s).zfill(6)}.psv' for s in patient_list]
    for f in patient_list:
        df = load_single_file(os.path.join("training", f))
        final_df = pd.concat([final_df, df])
    final_df.to_csv('sepsis.csv',index=False)
create_final_table()

# Bar Chart

Let's revisit the [2019 PhysioNet Challenge](https://physionet.org/content/challenge-2019/1.0.0/) dataset we used in the last session, but this time we will look at data recorded across multiple hours.

In [ ]:
df = pd.read_csv('sepsis.csv')
df

Imagine that we wanted to figure out how many rows in the table are associated with each patient. One of the best ways of displaying this information is a ***bar chart***.

To make this graph, we will want to get two sets of data:
1. The independent variable, which is the ID of each patient
2. The dependent variable, which is the number of rows they have in the dataset

To get this data from a `Series`, we can use the handy `.value_counts()` method. The values in the output will be our dependent values, and the corresponding indices will be our independent values.

In [ ]:
# Counts the number of times each unique patient name appears in the Series
num_samples = df['PatientID'].value_counts()
print(num_samples)

We can plot this data using the `bar()` function in `matplotlib`. At minimum, we need to specify the independent variable and the dependent variable in that order. These arguments can be lists, `numpy` arrays, or `pandas` `Series`.

The two other commands we're using below are not strictly necessary, but we'll use them to make the graphs look more tidy in Jupyter.

In [ ]:
plt.figure(figsize=(5, 3)) # Generates a new 3"x5" figure
plt.bar(num_samples.index, num_samples) # x-axis is Patient ID, y-axis is the number of samples
plt.show() # Shows the figure without any extra text in the console

This is a good starting point, but there are a couple of issues with this graph:
1. It's hard to read the patient names along the x-axis.
2. If we were to send this image to someone, they won't know what this graph is showing unless we give them some additional background information.

It is good to get in the habit of making clearly labelled graphs where people know exactly what they are looking at.

Let's clean up this image a bit.

In [ ]:
plt.figure(figsize=(5, 3))
plt.bar(num_samples.index, num_samples)
plt.xlabel('Patient ID') # Add a label to the x-axis
plt.ylabel('Num Samples') # Add a label to the y-axis
plt.title('Distribution of Data Availability') # Add a title
plt.xticks(rotation = 90) # We're rotating the tick marks so that they don't overlap
plt.show()

Notice that we created yet a new figure at the top of the previous code block. As we'll see later, this ensures that you are working with a blank canvas whenever you are creating a plot. Otherwise, you might generate two plots on the same figure instance.

# Box Plot

Now let's make this a bit more complicated. Rather than trying to plot a single number for each patient, let's look at the distribution of heart rates for each patient in our dataset. In other words, we want a ***box plot*** (i.e., a box-and-whiskers plot) showing the heart rate data of each patient.

To make this graph, we will want to get two sets of data:
1. The independent variable, which is the ID of each patient
2. The dependent variable, which is the corresponding list of heart rate measurements for each patient

There are two ways we can get these values. The first way will be manual, while the second will take full advantage of `pandas`. The manual approach will involve the following steps:
1. Initialize a data structure where we will accumulate our results.
2. Get all the patient IDs
3. For each patient ID, grab all of the corresponding rows in the `DataFrame`
4. Add the values to our accumulator

In [ ]:
# Create a place to store this data
patient_hr_data = {}

# Iterate through unique PatientIDs
for patient_id in df['PatientID'].unique():
    # Grab their heart rate data
    patient_df = df[df['PatientID'] == patient_id]
    patient_hr_data[patient_id] = patient_df['HR'].values

patient_hr_data

The fancy approach will utilize a `pandas` method called `.groupby()`. This method separates a `DataFrame` into a collection of sub-`DataFrames` according to a specific attribute. The output of this method is a special object that is not printable on its own, but we can easily see its contents using a `for` loop. Let's see what this looks like when we group our data by `PatientID`:

In [ ]:
# Separate the DataFrame into separate groups according to PatientID
groupby_patient = df.groupby(by=['PatientID'])

# Show the first two entries of each group
for group in groupby_patient:
    patient = group[0]
    frame = group[1]
    print(patient)
    print(frame.iloc[:2])
    print('-----------')

Recall that some of the entries of our `DataFrame` are empty. When we were calculating the mean and standard deviation earlier, `pandas` was good about ignoring `np.nan` values and only performing calculations on real values. `matplotlib` doesn't do this, so we will need to remove those values ourselves using a handy method called `.dropna()`.

Now that we know what `.groupby()` is doing, let's take it a step further to save the results in a slightly more user-friendly way for us:

In [ ]:
# Create a place to store this data
patient_hr_data = {}

# Iterate through unique PatientIDs
for group in df.groupby(by=['PatientID']):
    patient = group[0]
    frame = group[1]

    # Grab their heart rate data
    patient_id = patient[0] # This is needed because groupby can use multiple indices
    patient_hr_data[patient_id] = frame['HR'].values

patient_hr_data

`boxplot()` is different from other graphing functions in `matplotlib` because it only takes in a single input: a list of lists that will be used to generate the distributions. Another important consideration about `boxplot()` is that it does not like it calculating distributions when there are `nan` values. Here is some code to get the data in a format suitable for `boxplot()`

In [ ]:
# Save patient IDs for later
patient_ids = list(patient_hr_data.keys())

# Extract a list of lists from the dictionary
patient_hr_data_list = list(patient_hr_data.values())
for i in range(len(patient_hr_data_list)):
    patient_hrs = patient_hr_data_list[i]
    patient_hr_data_list[i] = patient_hrs[~np.isnan(patient_hrs)].tolist()

patient_hr_data_list

Let's see what happens when we use it.

In [ ]:
# Plot the data
plt.figure(figsize=(5, 3))
plt.boxplot(patient_hr_data_list)
plt.xlabel('Patient ID')
plt.ylabel('Heart Rate')
plt.title('Heart Rate Measurements')
plt.show()

The default tick marks along the bottom are the numbers 1...N, where N is the number of distributions we want to show. We can give these tick marks more meaningful names as follows:

In [ ]:
# Generate a numeric sequence corresponding to the original tick marks
old_ticks = np.arange(len(patient_ids))+1

# Plot the data
plt.figure(figsize=(5, 3))
plt.boxplot(patient_hr_data_list)
plt.xlabel('Patient ID')
plt.ylabel('Heart Rate')
plt.title('Heart Rate Measurements')
plt.xticks(ticks=old_ticks, labels=patient_ids, rotation=90)
plt.show()

# Line Graph

Finally, let's imagine that we wanted to create a ***line graph*** showing the changes in a patient's heart rate over time. To make this graph, we will want to get two sets of data:
1. The independent variable, which is the timestamp of each measurement
2. The dependent variable, which is the heart rate measurement at each timestamp.


In [ ]:
# Get the timestamps and measurements of the first patient's heart rate data
p1_data = df[df['PatientID'] == 'p000001']
p1_hr = p1_data['HR']
p1_hour = p1_data['Hour']

We can call the `plot()` function to generate a line graph of this data. Similar to what we was required for the `bar()` function, we need to at least specify the independent and dependent values in that order.

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot(p1_hour, p1_hr)
plt.title('p000001 Heart Rate Data')
plt.xlabel('Time (hr)')
plt.ylabel('Heart rate (bpm)')
plt.show()

Notice that this graph has gaps. This is because the heart rate value at that time was `np.nan`, and `matplotlib` ignores these values when generating plots.

Now let's say that we wanted to overlay the heart rate data from another patient on top of the graph. Let's start by grabbing that data.

In [ ]:
# Get the second patient's data
p2_data = df[df['PatientID'] == 'p000002']
p2_hr = p2_data['HR']
p2_hour = p2_data['Hour']

And now let's add that data to our graph. We can do that by calling `plot()` a second time without generating a new figure with `plt.figure()`.

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot(p1_hour, p1_hr)
plt.plot(p2_hour, p2_hr) # Plot the new data
plt.title('p000001 + p000002 Heart Rate Data')
plt.xlabel('Time (hr)')
plt.ylabel('Heart rate (bpm)')
plt.show()

So now we have two lines, and `matplotlib` automatically drew the new line with a different color. But how are other people going to know which line belongs to which patient? Let's add an informative legend.

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot(p1_hour, p1_hr, label='p000001') # Add a label to the first patient's data
plt.plot(p2_hour, p2_hr, label='p000002') # Add a label to the second patient's data
plt.title('Patient Heart Rate Data')
plt.xlabel('Time (hr)')
plt.ylabel('Heart rate (bpm)')
plt.legend() # Generate the legend
plt.show()

# Aesthetics

With all of the graphs we generated, there are hundreds of parameters we can leverage to tweak the appearance of the graph. `matplotlib` can change, among other visual attributes:
* Line color
* Line style (e.g., solid, dashed)
* Line thickness
* Points or no points
* Size of points

We don't have time to go over all of these features, but here is a quick example to show what is possible.

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot(p1_hour, p1_hr, 'k--o', label='p000001') # k = black, -- = dashed line, o = circles for points
plt.plot(p2_hour, p2_hr, 'r-*', label='p000002') # r = red, - = solid line, * = stars for points
plt.grid() # Add grid lines that match the tick marks along the axes
plt.title('Patient Heart Rate Data')
plt.xlabel('Time (hr)')
plt.ylabel('Heart rate (bpm)')
plt.legend()
plt.show()

# Subplots

You might find it helpful to put multiple plots side-by-side. In these situations, you can use the function `plt.subplot()`, which helps you arrange your plots in a grid. This function takes three arguments: (1) the number of rows you want to set up in your grid, (2) the number of columns you want to set up in your grid, and (3) the position ID of the current graph that you want to plot. For example, this code cell generates a 2 $\times$ 3 grid of empty plots:

In [ ]:
plt.figure(figsize=(7, 7))
plt.subplot(2, 3, 1)
plt.title('(2, 3, 1)')
plt.subplot(2, 3, 2)
plt.title('(2, 3, 2)')
plt.subplot(2, 3, 3)
plt.title('(2, 3, 3)')
plt.subplot(2, 3, 4)
plt.title('(2, 3, 4)')
plt.subplot(2, 3, 5)
plt.title('(2, 3, 5)')
plt.subplot(2, 3, 6)
plt.title('(2, 3, 6)')
plt.show()

You can basically think of these `plt.subplot()` calls as mini-`plt.figure()` calls. Once you create a new subplot, all of the new plots generated thereafer are added to that part of your overall figure.

In [ ]:
plt.figure(figsize=(7, 7))
plt.subplot(2, 3, 1)
plt.plot(p1_hour, p1_hr)
plt.title('(2, 3, 1): Patient 1 Data')
plt.subplot(2, 3, 2)
plt.title('(2, 3, 2)')
plt.subplot(2, 3, 3)
plt.title('(2, 3, 3): Patient 2 Data')
plt.plot(p2_hour, p2_hr)
plt.subplot(2, 3, 4)
plt.title('(2, 3, 4)')
plt.subplot(2, 3, 5)
plt.title('(2, 3, 5)')
plt.subplot(2, 3, 6)
plt.title('(2, 3, 6)')
plt.show()

You can actually get quite creative with subplots by mixing your grid specifications. In this example here, we split up the top row into two plots, but we pretend that the bottom row only has one spot so that the plot fills in both columns:

In [ ]:
plt.figure(figsize=(5, 7))
plt.subplot(2, 2, 1)
plt.title('(2, 2, 1)')
plt.subplot(2, 2, 2)
plt.title('(2, 2, 2)')
plt.subplot(2, 1, 2)
plt.title('(2, 1, 2)')
plt.show()